In [ ]:
# get started you must mount google colab to you driver
from google.colab import drive
drive.mount('/content/gdrive')
# if a request appear, please accept it.

In [ ]:
#unzip the Xraydata 
!unzip "/content/gdrive/yourdirve/datazipfile.zip" -d "/yourdrive"

In [ ]:
# import these library request
import cv2
import pickle
from os import listdir
import numpy as np
from sklearn.preprocessing import LabelBinarizer
from keras.preprocessing.image import ImageDataGenerator
from keras.callbacks import ModelCheckpoint
from keras.layers import Dense
from keras.applications.resnet import ResNet50
import matplotlib.pyplot as plt
import tensorflow as tf
from keras.models import Sequential
from sklearn.model_selection import train_test_split
from google.colab.patches import cv2_imshow


In [ ]:
# create data file function for training procession
# 'src' is the source image
def createDataFile(src):
    image = []
    label = []
    for folder in listdir(src):
      for picture in listdir(src + "/" +folder):
        img = cv2.imread(src +"/"+ folder + "/" + picture )
        img = cv2.resize(img, dsize=(224,224))
        image.append(img)
        label.append(folder)

    image = np.array(image)
    image = image.reshape(image.shape[0], 224, 224, 3)
    
    tobinary = LabelBinarizer()
    label = tobinary.fit_transform(label)
    label = np.array(label)

    fileData = open("dataTrain.data", 'wb')
    pickle.dump((image, label), fileData)
    fileData.close()


In [ ]:
#running createdata function
createDataFile("/content/gdrive/yourfolderContainImages")

In [ ]:
#define model use pretrain with Imagenet weights 


def model():
    model = Sequential()
    model.add(ResNet50(include_top=False, weights='imagenet', input_shape=(224, 224, 3), pooling='avg'))
    model.add(Dense(1, activation='sigmoid'))
    #You must disable Train bottom layers by the this command
    model.layers[0].trainable = False
    model.compile(loss = tf.keras.losses.BinaryCrossentropy(),
                  optimizer='adam',
                  metrics=['accuracy'])

    return model


In [ ]:
#create model and wait a few minute for download the pretrain weight
model = model()

In [ ]:
# Now you can load the data from dataTrain.data file by 'src' director
def readData(src):
  
    fileData = open(src, 'rb')

    image, label = pickle.load(fileData)
    fileData.close()
    return image, label

image, label = readData('/content/gdrive/dataTrain.data')
print(image.shape)
print(label.shape)

imageTrain, imageTest, labelTrain, labelTest = train_test_split(image, label, test_size=0.2, random_state=42)
print(imageTrain.shape)
print(imageTest.shape)

In [ ]:
#save better weight 
Weights = "weight_{epoch:02d}_{val_accuracy:.2f}.hdf5"
checkPoint = ModelCheckpoint(Weights, monitor='val_accuracy', verbose=1, save_best_only=True, mode='max')
callBacks = [checkPoint]

augment = ImageDataGenerator()


In [ ]:
# Train with 5 epochs
epochs = 5
train_result = model.fit_generator(augment.flow(imageTrain, labelTrain, batch_size=32),
                               epochs=epochs ,
                               validation_data=augment.flow(imageTest, labelTest,
                               batch_size=32),
                               callbacks=callBacks)

In [ ]:
#after trained model you can plot the accuracy score and lost score 
accuracy_train = train_result.history['accuracy']
accuracy_val =train_result.history['val_accuracy']
number_epochs = range(1,epochs+1)
plt.plot(number_epochs, accuracy_train, 'g', label='Training accuracy')
plt.plot(number_epochs, accuracy_val, 'b', label='validation accuracy')
plt.title('Training and Validation accuracy')
plt.xlabel('Epochs')
plt.ylabel('accuarcy')
plt.legend()
plt.show()

In [ ]:
loss_train = train_result.history['loss']
loss_val = train_result.history['val_loss']
epochs = range(1,epochs+1)
plt.plot(epochs, loss_train, 'g', label='Training loss')
plt.plot(epochs, loss_val, 'b', label='validation loss')
plt.title('Training and Validation loss')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.legend()
plt.show()

In [ ]:
#good luck